In [1]:
import pandas as pd
from scipy.io import arff
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import BernoulliNB,GaussianNB
import pickle
from sklearn.metrics import confusion_matrix,precision_score,recall_score,f1_score,accuracy_score

In [2]:
pe_train_df=pd.read_csv(r'../permission_emb_train.csv')
pe_train_df.head(2)

,App,com.mxmobile.mx20120309.permission.C2D_MESSAGE,COM.FSS.LOCK,com.wWebLoicbyalfred.permission.C2D_MESSAGE,android.permission.BIND_WALLPAPER,action_loadback,com.anydown.wjbb.SYSINSTALL_APK,com.wadventureques.permission.C2D_MESSAGE,android.intent.action.VIEW,COM.FSS.INS,...,emb24,emb25,emb26,emb27,emb28,emb29,emb30,emb31,emb32,emb33
0,04935f30d9a32b04ef9f65ebaf75d50eef1e9c11e0a97f...,0,0,0,0,0,0,0,0,0,...,0.293282,0.303603,0.382468,0.402910,0.404957,0.389828,0.213044,-0.464500,0.541421,1
1,f6fd5e739087d6f11daab8b2d59319f07d224f7195ab1b...,0,0,0,0,0,0,0,0,0,...,-0.671232,-0.654310,-0.632948,-0.750866,-0.589620,-0.637099,-0.743500,0.467523,-0.590537,0


In [3]:
pe_test_df=pd.read_csv(r'../permission_emb_test.csv')
pe_test_df.head(2)

,App,com.mxmobile.mx20120309.permission.C2D_MESSAGE,COM.FSS.LOCK,com.wWebLoicbyalfred.permission.C2D_MESSAGE,android.permission.BIND_WALLPAPER,action_loadback,com.anydown.wjbb.SYSINSTALL_APK,com.wadventureques.permission.C2D_MESSAGE,android.intent.action.VIEW,COM.FSS.INS,...,emb24,emb25,emb26,emb27,emb28,emb29,emb30,emb31,emb32,emb33
0,f715de1e4df2c0a91e1c9a61c9f31864a6b24c207a10be...,0,0,0,0,0,0,0,0,0,...,-0.315191,-0.364762,-0.241296,-0.390212,-0.271694,-0.353278,-0.447053,0.152264,-0.270066,0
1,e0f4a0180e0c559d572ce7b1ae581051cd34e2653cb573...,0,0,0,0,0,0,0,0,0,...,-0.089483,-0.199287,-0.110638,-0.146773,-0.083816,-0.214742,-0.218759,0.044780,0.041495,0


In [4]:
X_train_permission=pe_train_df.iloc[:,1:353].values
X_train_embedding=pe_train_df.iloc[:,353:385].values

In [5]:
X_test_permission=pe_test_df.iloc[:,1:353]
X_test_embedding=pe_test_df.iloc[:,353:385]

In [6]:
Label_train = pe_train_df.iloc[:,-1:]
Label_test=pe_test_df.iloc[:,-1:]
Y_train= Label_train['emb33'].values.tolist()
Y_test= Label_test['emb33'].values.tolist()

In [9]:
clf = BernoulliNB(binarize=None)

In [13]:
clf.fit(X_train_permission, Y_train)

BernoulliNB(binarize=None)

In [11]:
from sklearn.metrics import confusion_matrix,precision_score,recall_score,f1_score,accuracy_score

In [21]:
permission_prob_train=clf.predict_proba(X_train_permission)
permission_prob_test=clf.predict_proba(X_test_permission)

In [22]:
pred=clf.predict(X_test_permission)

In [23]:
acc=accuracy_score(Y_test,pred)
print(acc)
print(confusion_matrix(Y_test,pred))
precision = precision_score(Y_test,pred)
print('Precision: %f' % precision)
recall = recall_score(Y_test,pred)
print('recall: %f' % recall)
f1score= f1_score(Y_test,pred)
print('f1score: %f' % f1score)

0.9254419677171407
[[14545   708]
 [  553  1107]]
Precision: 0.609917
recall: 0.666867
f1score: 0.637122


In [19]:
clf_emb =GaussianNB()
clf_emb.fit(X_train_embedding, Y_train)

GaussianNB()

In [25]:
emb_prob_train=clf_emb.predict_proba(X_train_embedding)
emb_prob_test=clf_emb.predict_proba(X_test_embedding)

In [29]:
probablity_combined=np.hstack((permission_prob_train,emb_prob_train))
probablity_combined_test=np.hstack((permission_prob_test,emb_prob_test))
print(probablity_combined)
print(probablity_combined.shape)


[[1.64932986e-004 9.99835067e-001 3.94502619e-126 1.00000000e+000]
 [9.99991022e-001 8.97779830e-006 6.54644539e-016 1.00000000e+000]
 [9.99989148e-001 1.08522640e-005 1.55082573e-081 1.00000000e+000]
 ...
 [9.99919106e-001 8.08940425e-005 1.00000000e+000 5.07455389e-037]
 [9.99846502e-001 1.53498489e-004 1.00000000e+000 3.49104570e-037]
 [9.99994480e-001 5.51955866e-006 1.00000000e+000 6.68720627e-025]]
(39462, 4)


In [27]:
clf_combined =GaussianNB()
clf_combined.fit(probablity_combined, Y_train)

GaussianNB()

In [30]:
pred_combined=clf_combined.predict(probablity_combined_test)
print(pred_combined)
acc=accuracy_score(Y_test,pred_combined)
print(acc)
print(confusion_matrix(Y_test,pred_combined))
precision = precision_score(Y_test,pred_combined)
print('Precision: %f' % precision)
recall = recall_score(Y_test,pred_combined)
print('recall: %f' % recall)
f1score= f1_score(Y_test,pred_combined)
print('f1score: %f' % f1score)

[0 0 1 ... 0 0 0]
0.9321823449417608
[[14272   981]
 [  166  1494]]
Precision: 0.603636
recall: 0.900000
f1score: 0.722612
